Студент Александр Иванов, факультет ИИ, user:2818070

Датасет одежды. 2 класса. "Обычная одежда" и "Камуфляжная одежда"
https://www.kaggle.com/imneonizer/normal-vs-camouflage-clothes

![1](https://www.pyimagesearch.com/wp-content/uploads/2020/04/fine_tune_resnet_dataset.jpg)

Загрузить архив с данными можно по ссылке https://yadi.sk/d/utrrcRLZttCmNg

In [1]:
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from pyimagesearch import config
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import ResNet50
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Утилита для подсчета графических файлов в директории
import os

image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")

def list_images(basePath, contains=None):
    # список графических файлов
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    for (rootDir, dirNames, filenames) in os.walk(basePath): 
        for filename in filenames:
            if contains is not None and filename.find(contains) == -1:
                continue

            ext = filename[filename.rfind("."):].lower()

            if validExts is None or ext.endswith(validExts):
                imagePath = os.path.join(rootDir, filename)
                yield imagePath

In [3]:
# Основные пути к данным
BASE_PATH = "camo_not_camo"

TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

In [4]:
TRAIN_SPLIT = 0.75
VAL_SPLIT = 0.1
CLASSES = ["camouflage_clothes", "normal_clothes"]

INIT_LR = 1e-4
BS = 32
NUM_EPOCHS = 20

MODEL_PATH = "camo_detector.model"

In [5]:
# Структура датасета, это 3 папки train, valid, test (внутри 2 папки(2 класса) с изображениями)
totalTrain = len(list(list_images(TRAIN_PATH)))
totalVal = len(list(list_images(VAL_PATH)))
totalTest = len(list(list_images(TEST_PATH)))

In [6]:
# Загрузка данных трэйн через утилиту Керас с разными настройками
trainAug = ImageDataGenerator(
    rotation_range=25,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest")

In [7]:
# Загрузка данных для валидации
valAug = ImageDataGenerator()

In [8]:
# Выбор среднего значения для аргументации
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [9]:
# Загрузка данных через генератор для обучающей, валидационной и тестровой выборок
trainGen = trainAug.flow_from_directory(
    TRAIN_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=True,
    batch_size=BS)

valGen = valAug.flow_from_directory(
    VAL_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=BS)

testGen = valAug.flow_from_directory(
    TEST_PATH,
    class_mode="categorical",
    target_size=(224, 224),
    color_mode="rgb",
    shuffle=False,
    batch_size=BS)

Found 10731 images belonging to 2 classes.
Found 1192 images belonging to 2 classes.
Found 3975 images belonging to 2 classes.


In [10]:
# Загрузка моедли ResNet-50 network
print("[INFO] preparing model...")
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

[INFO] preparing model...
94773248/94765736 [==============================] - 223s 2us/step


In [11]:
# Сконструируем "голову" нейросети на предобученных данных
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)

In [12]:
# Поместим "голову" в нашу модель
model = Model(inputs=baseModel.input, outputs=headModel)


In [13]:
# Заморозим веса, чтобы они не обучались
for layer in baseModel.layers:
    layer.trainable = False

In [14]:
# Компилируем модель
opt = Adam(lr=INIT_LR, decay=INIT_LR / NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [11]:
# Тренируем модель
print("[INFO] training model...")
H = model.fit(
    trainGen,
    steps_per_epoch=totalTrain // BS,
    validation_data=valGen,
    validation_steps=totalVal // BS,
    epochs=NUM_EPOCHS)

[INFO] training model...
Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 335 steps, validate for 37 steps
Epoch 1/20
335/335 [==============================] - 248s 741ms/step - loss: 0.1625 - accuracy: 0.9350 - val_loss: 0.1778 - val_accuracy: 0.9324
Epoch 2/20
335/335 [==============================] - 240s 715ms/step - loss: 0.0910 - accuracy: 0.9664 - val_loss: 0.1149 - val_accuracy: 0.9578
Epoch 3/20
335/335 [==============================] - 239s 714ms/step - loss: 0.0790 - accuracy: 0.9711 - val_loss: 0.1322 - val_accuracy: 0.9519
Epoch 4/20
335/335 [==============================] - 240s 718ms/step - loss: 0.0638 - accuracy: 0.9769 - val_loss: 0.1127 - val_accuracy: 0.9595
Epoch 5/20
335/335 [==============================] - 239s 714ms/step - loss: 0.0580 - accuracy: 0.9789 - val_loss: 0.1225 - val_accuracy: 0.9578
Epoch 6/20
335/335 [==============================] - 239s 714ms/step - loss

In [22]:
# Сбросим тестовый генератор и сделаем предсказания
print("[INFO] evaluating network...")
testGen.reset()
predIdxs = model.predict(testGen, steps=(totalTest // BS) + 1)

[INFO] evaluating network...


In [13]:
# для каждого изображения предскажем класс
predIdxs = np.argmax(predIdxs, axis=1)

# Посомтрим метрику
print(classification_report(testGen.classes, predIdxs, target_names=testGen.class_indices.keys()))

                    precision    recall  f1-score   support

camouflage_clothes       0.95      0.98      0.97      1968
    normal_clothes       0.98      0.95      0.97      2007

          accuracy                           0.97      3975
         macro avg       0.97      0.97      0.97      3975
      weighted avg       0.97      0.97      0.97      3975



In [14]:
# Сохраним модель на диск
print("[INFO] saving model...")
model.save(MODEL_PATH, save_format="h5")

[INFO] saving model...


In [89]:
# Построим график потерь
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
# plt.show()
plt.savefig('plot.png')

NameError: name 'H' is not defined

![2](plot.png)